In [ ]:
############################################# IMPORTING package ################################################
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter.ttk import *

from tkinter import messagebox as mess
import tkinter.simpledialog as tsd
import csv

import os
import numpy as np
import pandas as pd
from PIL import Image
import datetime
import time
from PIL import ImageTk,Image
import cv2 as cv
from glob import glob
import os
from utils.poincare import calculate_singularities
from utils.segmentation import create_segmented_and_variance_images
from utils.normalization import normalize
from utils.gabor_filter import gabor_filter
from utils.frequency import ridge_freq
from utils import orientation
from utils.crossing_number import calculate_minutiaes
from tqdm import tqdm
from utils.skeletonize import skeletonize


############################################# FUNCTIONS ################################################

def assure_path_exists(path):
    dir = os.path.dirname(path)
    if not os.path.exists(dir):
        os.makedirs(dir)


# ##################################################################################

def tick():
    time_string = time.strftime('%H:%M:%S')
    clock.config(text=time_string)
    clock.after(200, tick)


# ###################################################################################

def contact():
    mess._show(title='Contact us', message="Please contact us on : 'himanshudangwal99@gmail.com' ")



# ######################################################################################

def clear1(txt):
    txt.delete(0, 'end')
    # res = "1)Take Images  >>>  2)Save Profile"
    # message1.configure(text=res)

def clear2(txt2):
    txt2.delete(0, 'end')
    # res = "1)Take Images  >>>  2)Save Profile"
    # message1.configure(text=res)

########################################Scanner##################
def fingerprint_pipline(input_img):
    block_size = 16

    # pipe line picture re https://www.cse.iitk.ac.in/users/biometrics/pages/111.JPG
    # normalization -> orientation -> frequency -> mask -> filtering

    normalized_img = normalize(input_img.copy(), float(100), float(100))

    # color threshold
    # threshold_img = normalized_img
    # _, threshold_im = cv.threshold(normalized_img,127,255,cv.THRESH_OTSU)
    # cv.imshow('color_threshold', normalized_img); cv.waitKeyEx()

    # ROI and normalisation
    (segmented_img, normim, mask) = create_segmented_and_variance_images(normalized_img, block_size, 0.2)

    # orientations
    angles = orientation.calculate_angles(normalized_img, W=block_size, smoth=False)
    orientation_img = orientation.visualize_angles(segmented_img, mask, angles, W=block_size)

    # find the overall frequency of ridges in Wavelet Domain
    freq = ridge_freq(normim, mask, angles, block_size, kernel_size=5, minWaveLength=5, maxWaveLength=15)

    # create gabor filter and do the actual filtering
    gabor_img = gabor_filter(normim, angles, freq)

    # thinning oor skeletonize
    thin_image = skeletonize(gabor_img)

    # minutias
    minutias = calculate_minutiaes(thin_image)

    # singularities
    singularities_img = calculate_singularities(thin_image, angles, 1, block_size, mask)

    # visualize pipeline stage by stage
    output_imgs = [input_img, normalized_img, segmented_img, orientation_img, gabor_img, thin_image, minutias, singularities_img]
    for i in range(len(output_imgs)):
        if len(output_imgs[i].shape) == 2:
            output_imgs[i] = cv.cvtColor(output_imgs[i], cv.COLOR_GRAY2RGB)
    results = np.concatenate([np.concatenate(output_imgs[:4], 1), np.concatenate(output_imgs[4:], 1)]).astype(np.uint8)

    return results

def rem():
    img_dir = './sample_inputs/*'
    output_dir = './output/'
    def open_images(directory):
        images_paths = glob(directory)
        return np.array([cv.imread(img_path,0) for img_path in images_paths])

    images = open_images(img_dir)
    # image pipeline
    os.makedirs(output_dir, exist_ok=True)
    for i, img in enumerate(tqdm(images)):
        results = fingerprint_pipline(img)
        cv.imwrite(output_dir+str(i)+'.png', results)

def scanner():
    window3=tk.Tk()
    window3.geometry("900x720")
    window3.minsize(900, 720)
    window3.resizable(True, False)
    window3.title("Existing User")
    window3.configure(background='#262523')
    message5 = tk.Label(window3, text="Checking for user existance", fg="white", bg="#262523", width=20,
                        height=1, font=('times', 29, ' bold '))
    message5.place(x=200, y=30)
    time.sleep(2)

    rem()

def scanner2():
    window4=tk.Tk()
    window4.geometry("900x720")
    window4.minsize(900, 720)
    window4.resizable(True, False)
    window4.title("Existing User")
    window4.configure(background='#262523')
    message6 = tk.Label(window4, text="Checking for user existance", fg="white", bg="#262523", width=20,
                        height=1, font=('times', 29, ' bold '))
    message6.place(x=200, y=30)
    time.sleep(2)
#     rem()

    
    


######################################## USED STUFFS ############################################

global key
key = ''

ts = time.time()
date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
day, month, year = date.split("-")

mont = {'01': 'January',
        '02': 'February',
        '03': 'March',
        '04': 'April',
        '05': 'May',
        '06': 'June',
        '07': 'July',
        '08': 'August',
        '09': 'September',
        '10': 'October',
        '11': 'November',
        '12': 'December'
        }


######################################## GUI FRONT-END ###########################################


###################### Withdraw ################
def Withdraw():
    window1 = tk.Tk()
    window1.geometry("900x650")
    window1.minsize(900, 650)
    window1.maxsize(900, 650)
    window1.resizable(True, False)
    window1.title("Withdrawing")
    window1.configure(background='#262523')
    
    message5 = tk.Label(window1, text="Try Matching FingerPrint", fg="white", bg="#262523", width=20,
                        height=1, font=('times', 29, ' bold '))
    message5.place(x=200, y=30)
    
    takeImg3 = tk.Button(window1, text="Scan Fingerprint", command=scanner2, fg="white", bg="#00aeff", width=20,
                        height=1, font=('times', 15, ' bold '))
    takeImg3.place(x=330, y=400)


################## Recognise_to_check ##############################################
def Recognise_to_check():
    window2=tk.Tk()
    window2.geometry("900x720")
    window2.minsize(900, 720)
    window2.resizable(True, False)
    window2.title("Existing User")
    window2.configure(background='#262523')
    message4 = tk.Label(window2, text="Check Balance", fg="white", bg="#262523", width=20,
                        height=1, font=('times', 29, ' bold '))
    message4.place(x=200, y=30)

    ####################### textboxes #################3
    lbl = tk.Label(window2, text="Enter Pin", width=18, height=1, fg="white", bg="#262523", font=('times', 17, ' bold '))
    lbl.place(x=78, y=138)

    txt = tk.Entry(window2, width=32, fg="black", font=('times', 15, ' bold '))
    txt.place(x=400, y=138)

    lbl2 = tk.Label(window2, text="Enter Name", width=18, fg="white", bg="#262523", font=('times', 17, ' bold '))
    lbl2.place(x=78, y=223)

    txt2 = tk.Entry(window2, width=32, fg="black", font=('times', 15, ' bold '))
    txt2.place(x=400, y=223)


    ################## buttons #########################
    clearButton1 = tk.Button(window2, text="Clear", command=lambda: clear1(txt), fg="black", bg="#00aeff", width=11,
                            activebackground="white", font=('times', 11, ' bold '))
    clearButton1.place(x=704, y=138)

    clearButton2 = tk.Button(window2, text="Clear", command=lambda: clear2(txt2), fg="black", bg="#00aeff", width=11,
                             activebackground="white", font=('times', 11, ' bold '))
    clearButton2.place(x=704, y=223)

    takeImg = tk.Button(window2, text="Scan Fingerprint", command=scanner, fg="white", bg="#00aeff", width=20,
                        height=1, font=('times', 15, ' bold '))
    takeImg.place(x=330, y=400)

    window2.mainloop()


#################### Main Screen #########################
window = tk.Tk()
window.geometry("1280x720")
window.minsize(1280,720)
window.maxsize(1280,720)
window.resizable(True, False)
window.title("ATM MACHINE")
window.configure(background='#262523')

messageTop = tk.Label(window, text="Fingerprint Based ATM Recognition System", fg="white", bg="#262523", width=55,
                    height=1, font=('times', 29, ' bold '))
messageTop.place(x=40, y=30)


######## for showing current time  ##########################3
frame1 = tk.Frame(window, bg="#c4c6ce")
frame1.place(x=400, y=100, width=260, height=30)

frame2 = tk.Frame(window, bg="#c4c6ce")
frame2.place(x=640, y=100, width=200, height=30)

datef = tk.Label(frame1, text=day + "-" + mont[month] + "-" + year + "  |  ", fg="orange", bg="#262523", width=55,
                 height=1, font=('times', 20, ' bold '))
datef.pack(fill='both', expand=1)

clock = tk.Label(frame2, fg="orange", bg="#262523", width=55, height=1, font=('times', 22, ' bold '))
clock.pack(fill='both', expand=1 )
tick()

###################### Buttons #################################3
Recognise_to_witdraw = tk.Button(window, text="Withdraw", command=Withdraw, fg="black", bg="white", width=35,
                     height=5, font=('times', 17, ' bold '))
Recognise_to_witdraw.place(x=100, y=250)
Recognise_to_check = tk.Button(window, text="Balance", command=Recognise_to_check, fg="black", bg="white", width=35, height=5,
                     font=('times', 17, ' bold '))
Recognise_to_check.place(x=730, y=250)

##################### MENUBAR #################################

menubar = tk.Menu(window, relief='ridge')
filemenu = tk.Menu(menubar, tearoff=0)
# window.configure(menu=menubar)

##################### END ######################################

window.mainloop()

100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [05:02<00:00,  3.49s/it]
